In [ ]:
"""
FastAPI model server for PlacementStatus prediction.
Loads models/model.joblib which contains a sklearn Pipeline (preprocessor + classifier).
"""
import joblib
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel
import os

MODEL_PATH = os.getenv("MODEL_PATH", "models/model.joblib")

class InputPayload(BaseModel):
    CGPA: float
    Internships: int
    Projects: int
    Workshops_Certifications: int
    AptitudeTestScore: int
    SoftSkillsRating: float
    ExtracurricularActivities: str
    PlacementTraining: str
    SSC_Marks: int
    HSC_Marks: int

app = FastAPI(title="Placement Prediction")

@app.on_event("startup")
def load_model():
    global model
    model = joblib.load(MODEL_PATH)
    print("Model loaded from", MODEL_PATH)

@app.get("/health")
def health():
    return {"status": "ok"}

@app.post("/predict")
def predict(payload: InputPayload):
    # map incoming fields to dataframe column names expected by pipeline
    df = pd.DataFrame([{
        "CGPA": payload.CGPA,
        "Internships": payload.Internships,
        "Projects": payload.Projects,
        "Workshops/Certifications": payload.Workshops_Certifications,
        "AptitudeTestScore": payload.AptitudeTestScore,
        "SoftSkillsRating": payload.SoftSkillsRating,
        "ExtracurricularActivities": payload.ExtracurricularActivities,
        "PlacementTraining": payload.PlacementTraining,
        "SSC_Marks": payload.SSC_Marks,
        "HSC_Marks": payload.HSC_Marks
    }])
    pred = model.predict(df)[0]
    proba = model.predict_proba(df)[0,1]
    return {"prediction": int(pred), "probability": float(proba)}

# for local run:
# uvicorn app.main:app --host 0.0.0.0 --port 8000
